In [1]:
# 핵심 인물 찾기
users = [
    {"id":0, "name": "Hero"},
    {"id":1, "name": "Dunn"},
    {"id":2, "name": "Sue"},
    {"id":3, "name": "Chi"},
    {"id":4, "name": "Thor"},
    {"id":5, "name": "Clive"},
    {"id":6, "name": "Hicks"},
    {"id":7, "name": "Devin"},
    {"id":8, "name": "Kate"},
    {"id":9, "name": "Klein"}
]

friendship_pairs = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
                   (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

# id가 1인 사용자의 모든 친구를 찾으려면 friendship_pairs의 모든 쌍을 순회해야한다. 많은 쌍이 있다면 시간이 오래걸린다. 

# dict(key: users.id, value: 사용자의 모든 친구 목록) 만들기
friendships = {user["id"]: [] for user in users}

for i, j in friendship_pairs:
    friendships[i].append(j)
    friendships[j].append(i)

friendships

{0: [1, 2],
 1: [0, 2, 3],
 2: [0, 1, 3],
 3: [1, 2, 4],
 4: [3, 5],
 5: [4, 6, 7],
 6: [5, 8],
 7: [5, 8],
 8: [6, 7, 9],
 9: [8]}

In [2]:
# 네트워크상에서 각 사용자의 평균 연결 수는 몇개?
def number_of_friends(user):
    "user의 친구는 몇명일까?"
    user_id = user["id"]
    friend_ids = friendships[user_id]
    return len(friend_ids)


total_connections = sum(number_of_friends(user) for user in users)

total_connections  # 24

num_users = len(users)
avg_connections = total_connections / num_users

avg_connections

2.4

In [5]:
# 친구가 제일 많은 사람, 제일 적은 사람 순으로 내림 차순 정렬
# (user_id, number_of_friends)로 구성된 리스트 생성
num_friends_by_id = [(user["id"], number_of_friends(user)) for user in users]
# 정렬
num_friends_by_id.sort(key=lambda id_and_friends: id_and_friends[1], reverse=True)

num_friends_by_id
# 네트워크에서 중심 역할을 하는 사람을 찾았다. 위 처럼 계산한 것을 네트워크 용어로 "연결 중심성"이라고 부른다.
# 연결 중심성은 계산하기 쉽지만, 항상 기대하는 결과를 가져다 주지 않는다.

[(1, 3),
 (2, 3),
 (3, 3),
 (5, 3),
 (8, 3),
 (0, 2),
 (4, 2),
 (6, 2),
 (7, 2),
 (9, 1)]

In [10]:
# 데이터 과학자 추천하기
# 친구의 친구를 소개해준다. 각 사용자의 친구에 대해 그 친구의 친구들을 살펴보고 사용자의 모든 친구에 대해 똑같은 작업을 반복하고 결과를 저장

def foaf_ids_bad(user):
    # "foaf": 친구의 친구("friend of a friend")를 의미하는 약자이다.
    return [foaf_id for friend_id in friendships[user["id"]] for foaf_id in friendships[friend_id]]

print(foaf_ids_bad(users[0]))  # users[0] : "Hero"
print(friendships)

[0, 2, 3, 0, 1, 3]
{0: [1, 2], 1: [0, 2, 3], 2: [0, 1, 3], 3: [1, 2, 4], 4: [3, 5], 5: [4, 6, 7], 6: [5, 8], 7: [5, 8], 8: [6, 7, 9], 9: [8]}


Hero도 자신의 친구의 친구이므로 사용자 0이 두번 포함되어 있다. 사용자1, 2도 0의 친구이므로 포함, 사용자 3은 사용자1,2의 친구이므로 역시 포함

In [14]:
# 서로가 함께 아는 친구(mutual friends) 몇명인지, 동시에 사용자가 이미 아는 사람을 제외하는 함수
from collections import Counter

def friends_of_friends(user):
    user_id = user["id"]
    return Counter(
        foaf_id
        for friend_id in friendships[user_id]  # 사용자의 친구 개개인에 대해
        for foaf_id in friendships[friend_id]  # 그들의 친구를 세어보고
        if foaf_id != user_id  # 사용자 자신과
        and foaf_id not in friendships[user_id]  # 사용자의 친구는 제외
    )
print(friends_of_friends(users[3]))

Counter({0: 2, 5: 1})


In [17]:
# 관심사 데이터 interests user_id, interest로 구성됨
interests = [(0, "Hadoop"), (0, "Big data"), (0, "HBase"),(0, "Java"),
            (0, "Spark"), (0, "Storm"), (0, "Cassandra"),
            (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
            (1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
            (2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"),
            (3, "statistics"), (3, "regression"), (3, "probability"),
            (4, "machine learning"), (4, "regression"), (4, "decision trees"),
            (4, "libsvm"),(5, "Python"),(5, "R"),(5, "Java"),(5, "C++"),
            (5, "Haskell"),(5, "programming languages"),(6, "statistics"),
            (6, "probability"),(6, "mathematics"),(6, "theory"),
            (7, "machine learning"),(7, "scikit-learn"),(7, "Mahout"),
            (7, "neural network"),(8, "neural network"),(8, "deep learning"),
            (8, "Big data"),(8, "artificial intelligence"),(9, "Hadoop"),
            (9, "Java"), (9, "MapReduce"), (9, "Bigdata")]

def data_scientists_who_like(target_interest):
    """특정 관심사를 갖고 있는 모든 사용자 id를 반환"""
    return [user_id for user_id, user_interest in interests if user_interest == target_interest]
# 코드를 호출할 때마다 관심사 데이터를 매번 처음부터 끝까지 훑어야 하는 단점 존재, 관심사로 사용자 index를 만들어 보자

In [20]:
from collections import defaultdict

# key: 관심사, value: 사용자 id
user_ids_by_interest = defaultdict(list)

for user_id, interest in interests:
    user_ids_by_interest[interest].append(user_id)

# Key: 사용자 id, value: 관심사
interests_by_user_id = defaultdict(list)

for user_id, interest in interests:
    interests_by_user_id[user_id].append(interest)
